### _Intern Rotation Duration Constraint_

This is the major point at which our current model falls apart.  We require interns to spend a period of time in each rotation in **consecutive** blocks.  Mathematically, the below system works, however it becomes flawed when trying to use it in CPLEX. This was an issue for two reasons:
1. The inability to input decision variables into "if" statements
    - This is something we have attemted to rectify with 'Big M' If/Then constraints.  This is currently still a work in progress, however we believe we have some issue with dependency of the variables - it would seem that the variables being dependent undermines the ability of If/Then to work correctly.
2. Our initial use of the $k$ values as strings. To combat the 2. issue, we attempted to change $k$ to an integer value, however this resulted in its inability to be used in the same fashion in $x_{jk}^i$. 
    - This has been resolved.
    


We are currently re-evaluating our options.  One that was explored was that we might just ascribe weights to each Rotation (this being the duration), and use a binary variable instead of $x_{ij}$.  However this limits us in the particular regard that we would not be able to specify specific weeks for availability - an aspect of flexibility integral as we refine the model.  There are to be a number of final constraints still to be added which depend on the model having this quality.

$$\sum_{\alpha = 0}^7 y_{1,k + \alpha}^i = 8 \; \text{  if  }\; x_{1,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{2,k + \alpha}^i = 4 \; \text{  if  }\; x_{2,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{3,k + \alpha}^i = 4 \; \text{  if  }\; x_{3,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{4,k + \alpha}^i = 4 \; \text{  if  }\; x_{4,k}^i = 1$$

$$\sum_{\alpha = 0}^1 y_{5,k + \alpha}^i = 2 \; \text{  if  }\; x_{5,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{6,k + \alpha}^i = 3 \; \text{  if  }\; x_{6,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{7,k + \alpha}^i = 3 \; \text{  if  }\; x_{7,k}^i = 1$$

$$\sum_{\alpha = 0}^1 y_{8,k + \alpha}^i = 2 \; \text{  if  }\; x_{8,k}^i = 1$$

$$\sum_{\alpha = 0}^3 y_{9,k + \alpha}^i = 4 \; \text{  if  }\; x_{9,k}^i = 1$$

$$\sum_{\alpha = 0}^2 y_{10,k + \alpha}^i = 3 \; \text{  if  }\; x_{10,k}^i = 1$$

$$\sum_{\alpha = 0}^4 y_{11,k + \alpha}^i = 5 \; \text{  if  }\; x_{11,k}^i = 1$$

$$ y_{12,k}^i = 1 \; \text{  if  }\; x_{12,k}^i = 1$$

$$ y_{13,k}^i = 1 \; \text{  if  }\; x_{13,k}^i = 1$$

$$ y_{14,k}^i = 1 \; \text{  if  }\; x_{14,k}^i = 1$$

$$ y_{15,k}^i = 1 \; \text{  if  }\; x_{15,k}^i = 1$$

$$ y_{16,k}^i = 1 \; \text{  if  }\; x_{16,k}^i = 1$$

It should be noted that were the above constraints valid in the model, additional decision variables would be needed.

### Below are some experimentations of duration expressions

Expanding one of these...

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y) \quad \forall_i \; \forall_{k=1}^{47}$$

$$8 - \bigg( w_{1,1}^1 + w_{1,2}^1 + \cdots + w_{1,8}^1 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^1 + w_{1,48}^1 + \cdots + w_{1,54}^1 \bigg) \leq M \cdot y$$
$$8 - \bigg( w_{1,1}^2 + w_{1,2}^2 + \cdots + w_{1,8}^2 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^2 + w_{1,48}^2 + \cdots + w_{1,54}^2 \bigg) \leq M \cdot y$$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$\vdots \quad \quad \quad \vdots \quad \quad \quad \vdots \quad \quad \quad \vdots $$
$$8 - \bigg( w_{1,47}^{11} + w_{1,48}^{11} + \cdots + w_{1,54}^{11} \bigg) \leq M \cdot y$$


$$ x_{1,1}^1 \leq M\cdot (1-y)$$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$ x_{1,47}^1 \leq M\cdot (1-y)$$
$$ x_{1,1}^2 \leq M\cdot (1-y)$$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$\vdots  \quad \quad \vdots \quad  \quad \vdots $$
$$ x_{1,47}^{11} \leq M\cdot (1-y)$$


There appears to be an issue here with having dependent aspects of the variables in the 'if' and 'then' statements - specifically.
In consequence we need an alternate formulation of the duration constraints.

In IBM CPLEX we currently have a constraint like this:

forall(i in Intern)
  sum(k in Week)x[i][1][k] >= 8;
  
However, this does not stipulate consecutive values.

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y_k \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y_k) \quad \forall_i \; \forall_{k=1}^{47}$$

_for intern $1$, rotation $1$, week $1$_

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 8 \cdot (1-y_k)$$
$$x_{1,1}^{1} \leq 1 \cdot y_k$$

when $y_1 = 0$

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 8$$
$$x_{1,1}^{1} \leq 0$$

when $y_1 = 1$

$$ 8- \bigg( w_{1,1}^1 + w_{1,2}^1 + w_{1,3}^1 + w_{1,4}^1 + w_{1,5}^1 + w_{1,6}^1 + w_{1,7}^1 + w_{1,8}^1 \bigg) \leq 0$$
$$x_{1,1}^{1} \leq 1$$

We need an additional constraint (for each system?) to limit the y.

$${\Large \forall}_{i=0}^{11} \; \sum_{k=1}^{47} \; \sum_{\alpha = 0}^7 \; C_{1}^{i} \cdot  y_{1,(k + \alpha)}^i = 8   $$

$$C_{1}^{1} \cdot  y_{1,(k + \alpha)}^1 = 8$$

$$ 8 - \sum_{\alpha = 0}^7 w_{1,k + \alpha}^i \leq M\cdot y \quad \forall_i \; \forall_{k=1}^{47}$$
$$ x_{1,k}^i \leq M\cdot (1-y) \quad \forall_i \; \forall_{k=1}^{47}$$